# 5. 수집된 데이터 탐색
## 5-1. 스파크 세션 생성
### 5-1-1. 스파크 객체를 생성하는 코드를 작성하고, Shift+Enter 로 스파크 버전을 확인합니다
<details><summary> 정답확인</summary>

> 스파크 엔진의 버전 `v3.1.2`이 출력되면 성공입니다

</details>
<br> 

In [1]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# 코어 스파크 라이브러리를 임포트 합니다
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName("Data Engineer Training Course")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
from IPython.display import display, display_pretty, clear_output, JSON
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size
spark

22/04/14 16:53:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### 3-0. 출력하기 연습
#### 3-0-1. notebook 에서는 셀의 가장 마지막 줄의 변수를 자동으로 출력합니다. 
- 예를 들어, `spark.sql("select * from user25")` 가 셀의 가장 마지막 줄에 있다면, 자동으로 데이터가 출력됩니다. 

In [35]:
spark.sql("select 1")

1
1


#### 3-0-2. 데이터프레임을 명시하지 않고 아래처럼 사용해도 바로 결과를 show 해서 볼 수 있습니다. 
- `spark.sql("show tables").show()`

In [36]:
spark.sql("select 1").show()

+---+
|  1|
+---+
|  1|
+---+



#### 3-0-3. 데이터프레임으로 연산한 결과를 파이썬 변수에 저장하기

- 지표 연산을 위해 앞에서 생성한 지표를 사용해야할 경우가 있습니다. 이 경우, 파이썬 변수에 저장해서 사용하는 것이 필요합니다. 
- dataframe 의 collect 를 사용해서 레코드인 Row 를 반환받아 특정 column 에 접근해 저장합니다. 
https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.collect.html

예를 들어 아래와 같이 dau 이라는 이름의 dataframe 이 있을 경우, 

| dau | count |
|-----|-------|
| 5   | 3     |

0번째(첫번째) row 의 'dau' column 의 값인 '5' 를 v_dau 라는 변수로 저장할 수 있습니다. 

```python
v_dau = dau.collect()[0]["DAU"]
```

### 5-1-2. 수집된 테이블을 데이터프레임으로 읽고, 스키마 및 데이터 출력하기
> 파케이 포맷의 경우는 명시적인 스키마 정의가 되어 있지만, Json 포맷의 경우는 데이터의 값에 따라 달라질 수 있기 때문에 데이터를 읽어들일 때에 주의해야 합니다

- 데이터프레임 변수명 및 경로는 아래와 같습니다
  - 2020/10/25 일자 고객 (parquet) : user25 <- user/20201025
  - 2020/10/25 일자 매출 (parquet) : purchase25 <- purchase/20201025
  - 2020/10/25 일자 접속 (json) : access25 <- access/20201025
- 아래의 제약조건을 만족 시켜야 합니다
  - 입력 포맷이 Json 인 경우는 json 명령어와 추정(infer) 옵션을 사용하세요 spark.read.option("inferSchema", "true").json("access/20201024")
  - 모든 데이터에 대한 스키마를 출력하세요 dataFrame.printSchema()
  - 데이터 내용을 출력하여 확인하세요 dataFrame.show() 혹은 display(dataFrame)
- 고객 정보 파일을 읽고, 스키마와 데이터 출력하기

<details><summary> 정답확인</summary>

> 고객, 매출 및 접속 데이터의 스키마와, 데이터가 모두 출력되면 성공입니다

</details>
<br>

In [2]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
user25 = spark.read.parquet("user/20201025")
user25.printSchema()
user25.show(truncate=False)
display(user25)

purchase25 = spark.read.parquet("purchase/20201025")
purchase25.createOrReplaceTempView("purchase25")
purchase25.printSchema()
display(purchase25)

accesslog = spark.read.option("inferSchema", "true").json("access/20201025")
accesslog.createOrReplaceTempView("accesslog")
access25 = spark.sql("select a_id, a_tag, a_time, a_timestamp, a_uid from accesslog")
access25.createOrReplaceTempView("access25")
access25.printSchema()
display(access25)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)
 |-- u_signup: integer (nullable = true)

+----+----------+--------+--------+
|u_id|u_name    |u_gender|u_signup|
+----+----------+--------+--------+
|1   |정휘센    |남      |20201025|
|2   |김싸이언  |남      |20201025|
|3   |박트롬    |여      |20201025|
|4   |청소기    |남      |20201025|
|5   |유코드제로|여      |20201025|
+----+----------+--------+--------+



u_id,u_name,u_gender,u_signup
1,정휘센,남,20201025
2,김싸이언,남,20201025
3,박트롬,여,20201025
4,청소기,남,20201025
5,유코드제로,여,20201025


root
 |-- p_time: string (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



p_time,p_uid,p_id,p_name,p_amount
1603571550,1,2000,LG DIOS,2000000
1603614755,1,2000,LG Gram,1800000
1603593500,2,2001,LG Cyon,1400000
1603572155,3,2002,LG TV,1000000
1603585955,5,2004,LG Gram,3500000
1603586155,5,2004,LG TV,2500000


root
 |-- a_id: string (nullable = true)
 |-- a_tag: string (nullable = true)
 |-- a_time: long (nullable = true)
 |-- a_timestamp: string (nullable = true)
 |-- a_uid: long (nullable = true)



a_id,a_tag,a_time,a_timestamp,a_uid
logout,access,1603570200,2020-10-25 05:10:00.000,2
logout,access,1603589500,2020-10-25 10:31:40.000,5
logout,access,1603579200,2020-10-25 07:40:00.000,3
logout,access,1603567200,2020-10-25 04:20:00.000,1
logout,access,1603584200,2020-10-25 09:03:20.000,4
login,access,1603577200,2020-10-25 07:06:40.000,3
login,access,1603586500,2020-10-25 09:41:40.000,5
login,access,1603573200,2020-10-25 06:00:00.000,2
login,access,1603584500,2020-10-25 09:08:20.000,4
login,access,1603569200,2020-10-25 04:53:20.000,2


## 5-2. 수집된 고객, 매출 및 접속 임시 테이블 생성
### 5-2-1. 데이터프레임을 이용하여 임시테이블 생성하기
- 각각 user25, purchase25, access25 으로 생성하세요.  

<br>
<details><summary> 정답확인</summary>

> `show tables` 결과로 `user25`, `purchase25`, `access25` 3개 테이블이 출력되면 성공입니다

</details>
<br>


In [4]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
user25.createOrReplaceTempView("user25")
purchase25.createOrReplaceTempView("purchase25")
access25.createOrReplaceTempView("access25")
spark.sql("show tables '*25'")

database,tableName,isTemporary
,access25,true
,purchase25,true
,user25,true


## 5-3. SparkSQL을 이용하여 테이블 별 데이터프레임 생성하기
### 5-3-1. 아래에 비어있는 조건을 채워서 올바른 코드를 작성하세요
* 아래의 조건이 만족되어야 합니다
  - 2020/10/25 에 등록(`u_signup`)된 유저만 포함될 것 <kbd>`u_signup` >= '20201025' and `u_signup` < '20201026'</kbd>
  - 2020/10/25 에 발생한 매출(`p_time`)만 포함할 것 <kbd>`p_time` >= '2020-10-25 00:00:00' and `p_time` < '2020-10-26 00:00:00'</kbd>

In [5]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
u_signup_condition = "u_signup >= '20201025' and u_signup < '20201026'"
user = spark.sql("select u_id, u_name, u_gender from user25").where(u_signup_condition)
user.createOrReplaceTempView("user")
display(user)

p_time_condition = "p_time >= '2020-10-25 00:00:00' and p_time < '2020-10-26 00:00:00'"
purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase25").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")
display(purchase)

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access25")
access.createOrReplaceTempView("access")
display(access)

spark.sql("show tables")

u_id,u_name,u_gender
1,정휘센,남
2,김싸이언,남
3,박트롬,여
4,청소기,남
5,유코드제로,여


p_time,p_uid,p_id,p_name,p_amount
2020-10-25 05:32:30,1,2000,LG DIOS,2000000
2020-10-25 17:32:35,1,2000,LG Gram,1800000
2020-10-25 11:38:20,2,2001,LG Cyon,1400000
2020-10-25 05:42:35,3,2002,LG TV,1000000
2020-10-25 09:32:35,5,2004,LG Gram,3500000
2020-10-25 09:35:55,5,2004,LG TV,2500000


a_id,a_tag,a_timestamp,a_uid
logout,access,2020-10-25 05:10:00.000,2
logout,access,2020-10-25 10:31:40.000,5
logout,access,2020-10-25 07:40:00.000,3
logout,access,2020-10-25 04:20:00.000,1
logout,access,2020-10-25 09:03:20.000,4
login,access,2020-10-25 07:06:40.000,3
login,access,2020-10-25 09:41:40.000,5
login,access,2020-10-25 06:00:00.000,2
login,access,2020-10-25 09:08:20.000,4
login,access,2020-10-25 04:53:20.000,2


database,tableName,isTemporary
,access,true
,access25,true
,accesslog,true
,purchase,true
,purchase25,true
,user,true
,user25,true


## 5-4. 생성된 테이블을 SQL 문을 이용하여 탐색하기
> SQL 혹은 DataFrame API 어느 쪽을 이용하여도 무관하며, 결과만 동일하게 나오면 정답입니다

### 5-4-1. 성별 출력
- 성별이 '남'인 목록을 출력하세요. 
- 성별이 '여'인 목록을 출력하세요

<details><summary> 정답확인</summary>

> 여 가 2 row, 남은 3 row 가 출력되면 정답입니다

    
+----+--------+--------+  
|u_id|  u_name|u_gender|  
+----+--------+--------+  
|   1|  정휘센|      남|  
|   2|김싸이언|      남|  
|   4|  청소기|      남|  
+----+--------+--------+  

+----+----------+--------+  
|u_id|    u_name|u_gender|  
+----+----------+--------+  
|   3|    박트롬|      여|  
|   5|유코드제로|      여|  
+----+----------+--------+  
  
</details>
<br>

In [53]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe user")
spark.sql("select * from user")
whereCondition_m = "u_gender = '남'"
male = spark.sql("select * from user").where(whereCondition_m)
male.show()

whereCondition_f = "u_gender = '여'"
female = spark.sql("select * from user").where(whereCondition_f)
female.show()

+----+--------+--------+
|u_id|  u_name|u_gender|
+----+--------+--------+
|   1|  정휘센|      남|
|   2|김싸이언|      남|
|   4|  청소기|      남|
+----+--------+--------+

+----+----------+--------+
|u_id|    u_name|u_gender|
+----+----------+--------+
|   3|    박트롬|      여|
|   5|유코드제로|      여|
+----+----------+--------+



### 5-4-2. 상품금액이 200만원을 초과하는 매출 목록을 출력하세요


<details><summary> 정답확인</summary>

|              p_time | p_uid | p_id |  p_name | p_amount |
|--------------------:|------:|-----:|--------:|---------:|
| 2020-10-25 09:32:35 |     5 | 2004 | LG Gram |  3500000 |
| 2020-10-25 09:35:55 |     5 | 2004 |   LG TV |  2500000 |

  
</details>
<br>

In [7]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe purchase")
spark.sql("select * from purchase")
selectClause = "select * from purchase where p_amount > 2000000"
spark.sql(selectClause)

p_time,p_uid,p_id,p_name,p_amount
2020-10-25 09:32:35,5,2004,LG Gram,3500000
2020-10-25 09:35:55,5,2004,LG TV,2500000


### 5-4-3. GroupBy 구문을 이용하여 로그인, 로그아웃 횟수를 출력하세요
<details><summary> 정답확인</summary>

|   a_id | count(a_id) | p_id |
|-------:|------------:|-----:|
| logout |           5 | 2004 |
|  login |           7 | 2004 |
  
</details>
<br>

In [8]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe access").show()
spark.sql("select * from access").show()
groupByClause="select a_id, count(a_id) from access group by a_id"
spark.sql(groupByClause)

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|       a_id|   string|   null|
|      a_tag|   string|   null|
|a_timestamp|   string|   null|
|      a_uid|   bigint|   null|
+-----------+---------+-------+

+------+------+--------------------+-----+
|  a_id| a_tag|         a_timestamp|a_uid|
+------+------+--------------------+-----+
|logout|access|2020-10-25 05:10:...|    2|
|logout|access|2020-10-25 10:31:...|    5|
|logout|access|2020-10-25 07:40:...|    3|
|logout|access|2020-10-25 04:20:...|    1|
|logout|access|2020-10-25 09:03:...|    4|
| login|access|2020-10-25 07:06:...|    3|
| login|access|2020-10-25 09:41:...|    5|
| login|access|2020-10-25 06:00:...|    2|
| login|access|2020-10-25 09:08:...|    4|
| login|access|2020-10-25 04:53:...|    2|
| login|access|2020-10-25 07:56:...|    4|
| login|access|2020-10-25 03:46:...|    1|
+------+------+--------------------+-----+



a_id,count(a_id)
logout,5
login,7


# 6. 기본 지표 생성
> 생성된 테이블을 통하여 기본 지표(DAU, DPU, DR, ARPU, ARPPU) 를 생성합니다
## 6-1. DAU (Daily Activer User) 지표를 생성하세요
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 지정한 일자의 접속한 유저 수 <kbd>count(distinct `a_uid`)</kbd>
  - 지표산식 : 지정한 일자의 접속 테이블에 로그(로그인 혹은 로그아웃)가 한 번 이상 발생한 이용자의 빈도수
  - 입력형태 : access 테이블
  - 출력형태 : number (컬럼명: DAU)

* v_dau 라는 파이썬 변수에 DAU 결과를 저장하세요.
  
<details><summary> 정답확인</summary>
  
> DAU 값이 5 가 나오면 정답입니다 
  
</details>
<br>


In [14]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
display(access)
distinctAccessUser = "select count(distinct a_uid) as DAU from access"
dau = spark.sql(distinctAccessUser)
display(dau)

v_dau = dau.collect()[0]["DAU"]
print(v_dau)

a_id,a_tag,a_timestamp,a_uid
logout,access,2020-10-25 05:10:00.000,2
logout,access,2020-10-25 10:31:40.000,5
logout,access,2020-10-25 07:40:00.000,3
logout,access,2020-10-25 04:20:00.000,1
logout,access,2020-10-25 09:03:20.000,4
login,access,2020-10-25 07:06:40.000,3
login,access,2020-10-25 09:41:40.000,5
login,access,2020-10-25 06:00:00.000,2
login,access,2020-10-25 09:08:20.000,4
login,access,2020-10-25 04:53:20.000,2


DAU
5


5


## 6-2. DPU (Daily Paying User) 지표를 생성하세요
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 지정한 일자의 구매 유저 수 <kbd>count(distinct `p_uid`)</kbd>
  - 지표산식 : 지정한 일자의 구매 테이블에 한 번이라도 구매가 발생한 이용자의 빈도수
  - 입력형태 : purchase 테이블
  - 출력형태 : number (컬럼명: PU)
* v_pu 라는 파이썬 변수에 PU 결과를 저장하세요.

<details><summary> 정답확인</summary>

> PU 값이 4 가 나오면 정답입니다

</details>
<br>



In [15]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

display(purchase)
distinctPayingUser = "select count(distinct p_uid) as PU from purchase"
pu = spark.sql(distinctPayingUser)
display(pu)
v_pu = pu.collect()[0]["PU"]
print(v_pu)

p_time,p_uid,p_id,p_name,p_amount
2020-10-25 05:32:30,1,2000,LG DIOS,2000000
2020-10-25 17:32:35,1,2000,LG Gram,1800000
2020-10-25 11:38:20,2,2001,LG Cyon,1400000
2020-10-25 05:42:35,3,2002,LG TV,1000000
2020-10-25 09:32:35,5,2004,LG Gram,3500000
2020-10-25 09:35:55,5,2004,LG TV,2500000


PU
4


4


## 6-3. DR (Daily Revenue) 지표를 생성하세요
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 지정한 일자에 발생한 총 매출 금액 <kbd>sum(`p_amount`)</kbd>
  - 지표산식 : 지정한 일자의 구매 테이블에 저장된 전체 매출 금액의 합
  - 입력형태 : access 테이블
  - 출력형태 : number (컬럼명: DR)
* v_dr 라는 파이썬 변수에 DR 결과를 저장하세요.

<details><summary> 정답확인</summary>

> DR 값이 12200000 이 나오면 정답입니다

</details>
<br> 

In [16]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
display(purchase)
sumOfDailyRevenue = "select sum(p_amount) as DR from purchase"
dr = spark.sql(sumOfDailyRevenue)
display(dr)
v_dr = dr.collect()[0]["DR"]
print(v_dr)

p_time,p_uid,p_id,p_name,p_amount
2020-10-25 05:32:30,1,2000,LG DIOS,2000000
2020-10-25 17:32:35,1,2000,LG Gram,1800000
2020-10-25 11:38:20,2,2001,LG Cyon,1400000
2020-10-25 05:42:35,3,2002,LG TV,1000000
2020-10-25 09:32:35,5,2004,LG Gram,3500000
2020-10-25 09:35:55,5,2004,LG TV,2500000


DR
12200000


12200000


## 6-4. ARPU (Average Revenue Per User) 지표를 생성하세요
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 유저 당 평균 발생 매출 금액
  - 지표산식 : 총 매출 / 전체 유저 수 = DR / DAU
  - 입력형태 : Daily Revenue, Daily Active User
  - 출력형태 : number (문자열: ARPU )

<details><summary> 정답확인</summary>

> ARPU 가 2440000.0 이 나오면 정답입니다

</details>
<br>


In [17]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# print("ARPU : {}".format(<유저당 매출 금액 계산식>))
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")

+------------------+
|             ARPU |
+------------------+
|        2440000.0 |
+------------------+


## 6-5. ARPPU (Average Revenue Per Paying User) 지표를 생성하세요
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 유저 당 평균 발생 매출 금액
  - 지표산식 : 총 매출 / 전체 유저 수 = DR / PU
  - 입력형태 : Daily Revenue, Daily Paying User
  - 출력형태 : number

<details><summary> 정답확인</summary>

> ARPPU 가 3050000.0 이 나오면 정답입니다

</details>
<br>

In [64]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")

+------------------+
|            ARPPU |
+------------------+
|        3050000.0 |
+------------------+


# 7. 고급 지표 생성
## 7-1. 디멘젼 테이블을 설계 합니다
* 아래의 조건을 만족해야 합니다
  - 지표정의 : 이용자 누적 상태 정보
  - 지표산식 : 오늘까지 접속한 모든 유저의 정보를 저장하는 테이블
  - 입력형태 : user, purchase, access
  - 출력형태 : 아래와 같이 설계합니다

* 디멘젼 테이블의 스키마는 아래와 같습니다

| 컬럼명 | 컬럼타입 | 설명 |
| --- | --- | --- |
| `d_uid` | integer | 아이디 |
| `d_name` | string | 이름 |
| `d_gender` | string | 성별 |
| `d_acount` | integer | 누적 접속 횟수 |
| `d_pamount` | integer | 누적 구매 금액 |
| `d_pcount` | integer | 누적 구매 횟수 |
| `d_first_purchase` | string | 최초 구매 일시 |

## 7-2. 접속한 모든 고객의 id 및 접속 횟수를 가진 데이터프레임을 생성합니다

* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 접속한 이용자 정보 <kbd>select `a_uid`, count(`a_uid`) ... group by `a_uid`</kbd>
  - 지표산식 : 접속 여부는 'login' 로그가 존재하면 접속한 유저로 가정
  - 출력형태 : `a_uid`, `a_count`
  - 정렬형태 : `a_uid` asc

* access 테이블로부터 `a_uid` 가 'login' 인 `a_uid` 값의 빈도수를 group by `a_uid` 집계를 통해 구하시오

<details><summary> 정답확인</summary>
    
    
| a_uid | a_count |
|------:|--------:|
|     1 |       1 |
|     2 |       2 |
|     3 |       1 |
|     4 |       2 |
|     5 |       1 |

</details>
<br>

In [20]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
access.printSchema()

countOfAccess = "select a_uid, count(a_uid) as a_count from access where a_id = 'login' group by a_uid"
accs = spark.sql(countOfAccess).orderBy(asc("a_uid"))
display(accs)

root
 |-- a_id: string (nullable = true)
 |-- a_tag: string (nullable = true)
 |-- a_timestamp: string (nullable = true)
 |-- a_uid: long (nullable = true)



a_uid,a_count
1,1
2,2
3,1
4,2
5,1


## 7-3. 이용자별 총 매출 금액과, 구매 횟수를 가지는 데이터프레임을 생성합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 이용자 별 매출 횟수와, 합 <kbd>select `p_uid`, count(`p_uid`), sum(`p_amount`) ... group by `p_uid`</kbd>
  - 지표산식 : 매출 금액을 이용자 별로 집계합니다
  - 입력형태 : purchase
  - 출력형태 : `p_uid`, `p_count`, `p_amount`
  - 정렬형태 : `p_count` desc`, p_amount` desc
* purchase 테이블로 부터 `p_uid` 별 매출 횟수(count)와, 매출 금액의 합(sum)을 구하는 집계 쿼리를 생성 하시오


<details><summary> 정답확인</summary>
    
| p_uid | p_amount | p_count |
|------:|---------:|--------:|
|     5 |  6000000 |       2 |
|     1 |  3800000 |       2 |
|     2 |  1400000 |       1 |
|     3 |  1000000 |       1 |
</details>
<br>


In [21]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
purchase.printSchema()
sumOfCountAndAmount = "select p_uid, sum(p_amount) as p_amount, count(p_uid) as p_count from purchase group by p_uid"
amts = spark.sql(sumOfCountAndAmount).orderBy(desc("p_count"),desc("p_amount"))
display(amts)

root
 |-- p_time: string (nullable = true)
 |-- p_uid: integer (nullable = true)
 |-- p_id: integer (nullable = true)
 |-- p_name: string (nullable = true)
 |-- p_amount: integer (nullable = true)



p_uid,p_amount,p_count
5,6000000,2
1,3800000,2
2,1400000,1
3,1000000,1


## 7-4. 이용자 정보와 구매 정보와 조인합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 이용자 데이터와 매출 정보의 결합 <kbd>leftSide.join(rightSide, joinCondition, joinHow)</kbd>
  - 지표산식 : 이용자는 반드시 존재해야하며, 매출은 없어도 됩니다 (`left_outer`)
  - 입력형태 : accs, amts
  - 출력형태 : `a_uid`, `a_count`, `p_uid`, `p_count`, `p_amount`
  - 정렬형태 : `a_uid` asc

* 7-2 에서 생성한 accs 와 7-3 에서 생성한 amts 데이터를 uid 값을 기준으로 left outer 조인을 수행합니다

<details><summary> 정답확인</summary>

> uid 가 4번인 이용자만 매출 정보가 null 이면 정답입니다

</details>
<br>

In [22]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
accs.printSchema()
amts.printSchema()
joinCondition = (accs.a_uid == amts.p_uid)
joinHow = "left_outer"
dim1 = accs.join(amts, joinCondition, joinHow)
dim1.printSchema()
display(dim1.orderBy(asc("a_uid")))

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)

root
 |-- p_uid: integer (nullable = true)
 |-- p_amount: long (nullable = true)
 |-- p_count: long (nullable = false)

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_uid: integer (nullable = true)
 |-- p_amount: long (nullable = true)
 |-- p_count: long (nullable = true)



a_uid,a_count,p_uid,p_amount,p_count
1,1,1,3800000,2
2,2,2,1400000,1
3,1,3,1000000,1
4,2,null,null,null
5,1,5,6000000,2


## 7-5. 고객 정보를 추가합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 고객의 이름과 성별을 추가합니다 
  - 지표산식 : dim1 테이블과 고객정보를 결합합니다
  - 입력형태 : dim1, user
  - 출력형태 : `a_uid`, `a_count`, `p_uid`, `p_count`, `p_amount`, `u_id`, `u_name`, `u_gender`
  - 정렬형태 : `a_uid` asc

* 7-4 에서 생성한 dim1 데이터와 기존의 user 데이터를 left outer 로 결합합니다

<details><summary> 정답확인</summary>

> uid 가 4번인 이용자만 매출 정보가 null 이면 정답입니다

</details>
<br>


In [23]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim1.printSchema()
user.printSchema()
joinCondition = (dim1.a_uid == user.u_id)
joinHow = "left_outer"
dim2 = dim1.join(user, joinCondition, joinHow)
dim2.printSchema()
display(dim2.orderBy(asc("a_uid")))

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_uid: integer (nullable = true)
 |-- p_amount: long (nullable = true)
 |-- p_count: long (nullable = true)

root
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_uid: integer (nullable = true)
 |-- p_amount: long (nullable = true)
 |-- p_count: long (nullable = true)
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



a_uid,a_count,p_uid,p_amount,p_count,u_id,u_name,u_gender
1,1,1,3800000,2,1,정휘센,남
2,2,2,1400000,1,2,김싸이언,남
3,1,3,1000000,1,3,박트롬,여
4,2,null,null,null,4,청소기,남
5,1,5,6000000,2,5,유코드제로,여


## 7-6. 중복되는 ID 컬럼은 제거하고, 숫자 필드에 널값은 0으로 기본값을 넣어줍니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 중복되는 uid 컬럼을 제거하고, 매출정보가 없는 계정은 0으로 기본값을 넣어줍니다
  - 지표산식 : <kbd>fillDefaultValue = {"`p_amount`":0, "`p_count`":0}</kbd>
  - 입력형태 : dim2
  - 출력형태 : `a_uid`, `a_count`, `p_amount`, `p_count`, `u_name`, `u_gender`
  - 정렬형태 : `a_uid` asc

* 7-4 에서 생성한 dim1 데이터와 user 데이터를 결합하되, 중복 컬럼은 제거하고, 기본값을 0으로 채우는 `fillDefaultValue`를 작성하세요

<details><summary> 정답확인</summary>

> uid 가 4번인 이용자만 매출 정보가 0 이면 정답입니다

</details>
<br>


In [82]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim2.printSchema()
dim3 = dim2.drop("p_uid", "u_id")
fillDefaultValue = {"p_amount":0, "p_count":0}
dim4 = dim3.na.fill(fillDefaultValue)
dim4.printSchema()
display(dim4.orderBy(asc("a_uid")))

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_uid: integer (nullable = true)
 |-- p_amount: long (nullable = true)
 |-- p_count: long (nullable = true)
 |-- u_id: integer (nullable = true)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_amount: long (nullable = false)
 |-- p_count: long (nullable = false)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



a_uid,a_count,p_amount,p_count,u_name,u_gender
1,1,3800000,2,정휘센,남
2,2,1400000,1,김싸이언,남
3,1,1000000,1,박트롬,여
4,2,0,0,청소기,남
5,1,6000000,2,유코드제로,여


## 7-7. 생성된 유저 테이블을 재사용 가능하도록 컬럼 명을 변경합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 컬럼명이 `d_`로 시작하도록 일괄 변경합니다
  - 지표산식 : <kbd>withColumnRenamed("`a_uid`", "`d_uid`" )</kbd>
  - 입력형태 : dim4
    - dimension 테이블이므로 `d_`로 시작하는 컬럼 규칙을 따릅니다
    - access, purchase 와 같이 개별 테이블의 prefix 를 이용해서 `d_a<column-name>` 혹은 `d_p<column-name>` 규칙을 따릅니다
  - 출력형태 : `d_uid`, `d_name`, `d_gender`, `d_acount`, `d_pamount`, `d_pcount`
  - 정렬형태 : `d_uid` asc

* 7-6 에서 생성한 dim4 의 모든 컬럼이 `d_`로 시작하도록 Rename 하여 정리합니다

<details><summary> 정답확인</summary>

> 모든 컬럼이 `d_`로 시작하고 6개의 컬럼으로 구성되어 있다면 정답입니다

</details>
<br>


In [88]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim4.printSchema()
dim5 = (
    dim4
    .withColumnRenamed("a_uid", "d_uid")
    .withColumnRenamed("a_count", "d_acount")
    .withColumnRenamed("p_amount", "d_pamount")
    .withColumnRenamed("p_count", "d_pcount")
    .withColumnRenamed("u_name", "d_name")
    .withColumnRenamed("u_gender", "d_gender")
   .select("d_uid", "d_name", "d_gender", "d_acount", "d_pamount", "d_pcount").orderBy(asc("d_uid"))
)


display(dim5)

root
 |-- a_uid: long (nullable = true)
 |-- a_count: long (nullable = false)
 |-- p_amount: long (nullable = false)
 |-- p_count: long (nullable = false)
 |-- u_name: string (nullable = true)
 |-- u_gender: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount
1,정휘센,남,1,3800000,2
2,김싸이언,남,2,1400000,1
3,박트롬,여,1,1000000,1
4,청소기,남,2,0,0
5,유코드제로,여,1,6000000,2


## 7-8. 최초 구매정보를 유저 정보에 추가합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 지표정의 : 최초 구매시간 정보를 추가합니다. 
    - 최초 구매정보가 없다면 null 로 두고, 그렇지 않으면 가장 과거 일자의 구매시간을 추가합니다.
  - 지표산식 : 해당 일자의 가장 처음 구매한 구매일시를 이용하여 최초구매일을 구합니다
    * 하루에 여러 번 구매가 있을 수 있으므로 가장 먼저 구매한 일시를 최초 구매시간으로 합니다.
    * selectFirstPurchaseTime : min 함수를 사용해 `p_time` 의 최소값을 구합니다
  - 입력형태 : dim5
  - 출력형태 : `d_uid`, `d_count`, `d_amount`, `d_count`, `d_name`, `d_gender`, `d_first_purchase`
  - 정렬형태 : `d_uid` asc

<details><summary> 정답확인</summary>

> 4번 고객의 제외한 모든 첫 번째 구매 일시가 출력되면 정답입니다

</details>
<br>


In [125]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# purchase.printSchema()
selectFirstPurchaseTime = "select p_uid, min(p_time) as d_first_purchase from purchase group by p_uid"
first_purchase = spark.sql(selectFirstPurchaseTime)
# print("--------first_purchase--------------")
# first_purchase.show()

dim7 = (
    dim5.join(first_purchase, (dim5.d_uid == first_purchase.p_uid), "left_outer")
    .drop("p_uid")
)
display(dim7)

dimension = dim7.orderBy(asc("d_uid"))
dimension.printSchema()
display(dimension)

d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
5,유코드제로,여,1,6000000,2,2020-10-25 09:32:35
1,정휘센,남,1,3800000,2,2020-10-25 05:32:30
3,박트롬,여,1,1000000,1,2020-10-25 05:42:35
2,김싸이언,남,2,1400000,1,2020-10-25 11:38:20
4,청소기,남,2,0,0,null


root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = false)
 |-- d_pamount: long (nullable = false)
 |-- d_pcount: long (nullable = false)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
1,정휘센,남,1,3800000,2,2020-10-25 05:32:30
2,김싸이언,남,2,1400000,1,2020-10-25 11:38:20
3,박트롬,여,1,1000000,1,2020-10-25 05:42:35
4,청소기,남,2,0,0,null
5,유코드제로,여,1,6000000,2,2020-10-25 09:32:35


## 7-9. 생성된 디멘젼을 저장소에 저장합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 저장위치 : "dimension/dt=20201025" <kbd>dataFrame.write</kbd>
  - 저장옵션 : 대상 경로가 존재하더라도 덮어씁니다 <kbd>.mode("overwrite")</kbd>
  - 저장포맷 : 파케이  <kbd>.parquet("dimension/dt=20201025")</kbd>
  
<details><summary> 정답확인</summary>

> 저장 시에 오류가 없고 대상 경로(dimension/dt=20201025)가 생성되었다면 성공입니다

</details>
<br>


In [126]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dimension.printSchema()
target_dir="dim_users/dt=20201025"
dimension.write.mode("overwrite").parquet(target_dir)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = false)
 |-- d_pamount: long (nullable = false)
 |-- d_pcount: long (nullable = false)
 |-- d_first_purchase: string (nullable = true)



## 7-10. 생성된 디멘젼을 다시 읽어서 출력합니다
* 아래의 조건이 만족하는 코드를 작성하세요
  - 저장위치 : "dimension/dt=20201025" <kbd>spark.read</kbd>
  - 저장포맷 : 파케이  <kbd>.parquet("dimension/dt=20201025")</kbd>

<details><summary> 정답확인</summary>

> 디멘젼 테이블을 정상적으로 읽어왔고, 동일한 스키마와 데이터가 출력되었다면 정답입니다

</details>
<br>


In [127]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
newDimension = spark.read.parquet(target_dir)
newDimension.printSchema()
display(newDimension)

root
 |-- d_uid: long (nullable = true)
 |-- d_name: string (nullable = true)
 |-- d_gender: string (nullable = true)
 |-- d_acount: long (nullable = true)
 |-- d_pamount: long (nullable = true)
 |-- d_pcount: long (nullable = true)
 |-- d_first_purchase: string (nullable = true)



d_uid,d_name,d_gender,d_acount,d_pamount,d_pcount,d_first_purchase
5,유코드제로,여,1,6000000,2,2020-10-25 09:32:35
2,김싸이언,남,2,1400000,1,2020-10-25 11:38:20
3,박트롬,여,1,1000000,1,2020-10-25 05:42:35
1,정휘센,남,1,3800000,2,2020-10-25 05:32:30
4,청소기,남,2,0,0,null


In [128]:
# 최종 결과가 아래의 값과 일치하는지 확인합니다
# DAU:5, PU:4, DR: 12200000, ARPU: 2440000.0, ARPPU: 3050000.0
print("+------------------+")
print("|              DAU |")
print("+------------------+")
print("|                {} |".format(v_dau))
print("+------------------+")
print("+------------------+")
print("|               PU |")
print("+------------------+")
print("|                {} |".format(v_pu))
print("+------------------+")
print("+------------------+")
print("|               DR |")
print("+------------------+")
print("|         {} |".format(v_dr))
print("+------------------+")
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")

+------------------+
|              DAU |
+------------------+
|                5 |
+------------------+
+------------------+
|               PU |
+------------------+
|                4 |
+------------------+
+------------------+
|               DR |
+------------------+
|         12200000 |
+------------------+
+------------------+
|             ARPU |
+------------------+
|        2440000.0 |
+------------------+
+------------------+
|            ARPPU |
+------------------+
|        3050000.0 |
+------------------+
